# Automate checking if theres free space in a class


In [1]:
import pandas as pd
import requests
import numpy as np

In [2]:
url = "https://classes.usc.edu/term-20221/course/jour-598/"

In [3]:
classes = pd.read_html(url)[0]

In [4]:
classes

,Section,Session,Type,Time,Days,Registered,Instructor,Location,Syllabus,Info
0,21621R,1,Lecture,9:00-12:20pm,Monday,6 of 11,NaN,ANN308,NaN,NaN
1,21622R,1,Lecture,9:00-12:20pm,Monday,3 of 11,NaN,ANN413,NaN,NaN
2,21623R,1,Lecture,12:30-3:50pm,Monday,11 of 11,"Alan Mittelstaedt,Lisa Pecot-Hebert",ANN408,NaN,NaN
3,21624R,1,Lecture,12:30-3:50pm,Monday,3 of 11,Alan Abrahamson,ANN307,NaN,NaN
4,21625R,1,Lecture,12:30-3:50pm,Monday,6 of 11,Vince Gonzales,ANN211,NaN,NaN


Interesting output...

Lets try to clean it up

## Split the registered column

In [5]:
classes["Registered"].str.split(n=2, expand=True)[0]

0     6
1     3
2    11
3     3
4     6
Name: 0, dtype: object

In [6]:
# Wonderful. Lets add it to the dataframe
classes["JC_Registered"] = classes["Registered"].str.split(n=2, expand=True)[0]

In [7]:
classes

,Section,Session,Type,Time,Days,Registered,Instructor,Location,Syllabus,Info,JC_Registered
0,21621R,1,Lecture,9:00-12:20pm,Monday,6 of 11,NaN,ANN308,NaN,NaN,6
1,21622R,1,Lecture,9:00-12:20pm,Monday,3 of 11,NaN,ANN413,NaN,NaN,3
2,21623R,1,Lecture,12:30-3:50pm,Monday,11 of 11,"Alan Mittelstaedt,Lisa Pecot-Hebert",ANN408,NaN,NaN,11
3,21624R,1,Lecture,12:30-3:50pm,Monday,3 of 11,Alan Abrahamson,ANN307,NaN,NaN,3
4,21625R,1,Lecture,12:30-3:50pm,Monday,6 of 11,Vince Gonzales,ANN211,NaN,NaN,6


Nice!

In [8]:
classes["Registered"].str.split(n=2, expand=True)[2]

0    11
1    11
2    11
3    11
4    11
Name: 2, dtype: object

In [9]:
# Wonderful. Lets add it to the dataframe
classes["JC_Capacity"] = classes["Registered"].str.split(n=2, expand=True)[2]

In [10]:
classes

,Section,Session,Type,Time,Days,Registered,Instructor,Location,Syllabus,Info,JC_Registered,JC_Capacity
0,21621R,1,Lecture,9:00-12:20pm,Monday,6 of 11,NaN,ANN308,NaN,NaN,6,11
1,21622R,1,Lecture,9:00-12:20pm,Monday,3 of 11,NaN,ANN413,NaN,NaN,3,11
2,21623R,1,Lecture,12:30-3:50pm,Monday,11 of 11,"Alan Mittelstaedt,Lisa Pecot-Hebert",ANN408,NaN,NaN,11,11
3,21624R,1,Lecture,12:30-3:50pm,Monday,3 of 11,Alan Abrahamson,ANN307,NaN,NaN,3,11
4,21625R,1,Lecture,12:30-3:50pm,Monday,6 of 11,Vince Gonzales,ANN211,NaN,NaN,6,11


## Create a Open? Column

def is_open(row):
    if row["JC_Registered"] == row["JC_Capacity"]:
        return "Closed"
    else:
        return "Open"

In [15]:
classes["JC_Open?"] = classes.apply(is_open, axis=1)

In [16]:
classes

,Section,Session,Type,Time,Days,Registered,Instructor,Location,Syllabus,Info,JC_Registered,JC_Capacity,JC_Open?
0,21621R,1,Lecture,9:00-12:20pm,Monday,6 of 11,NaN,ANN308,NaN,NaN,6,11,Open
1,21622R,1,Lecture,9:00-12:20pm,Monday,3 of 11,NaN,ANN413,NaN,NaN,3,11,Open
2,21623R,1,Lecture,12:30-3:50pm,Monday,11 of 11,"Alan Mittelstaedt,Lisa Pecot-Hebert",ANN408,NaN,NaN,11,11,Closed
3,21624R,1,Lecture,12:30-3:50pm,Monday,3 of 11,Alan Abrahamson,ANN307,NaN,NaN,3,11,Open
4,21625R,1,Lecture,12:30-3:50pm,Monday,6 of 11,Vince Gonzales,ANN211,NaN,NaN,6,11,Open


Nice!

## Remove unneeded cols

In [21]:
classes.drop("Syllabus", axis=1).drop("Info", axis=1).drop("Session", axis=1).drop("Type", axis=1)

,Section,Time,Days,Registered,Instructor,Location,JC_Registered,JC_Capacity,JC_Open?
0,21621R,9:00-12:20pm,Monday,6 of 11,NaN,ANN308,6,11,Open
1,21622R,9:00-12:20pm,Monday,3 of 11,NaN,ANN413,3,11,Open
2,21623R,12:30-3:50pm,Monday,11 of 11,"Alan Mittelstaedt,Lisa Pecot-Hebert",ANN408,11,11,Closed
3,21624R,12:30-3:50pm,Monday,3 of 11,Alan Abrahamson,ANN307,3,11,Open
4,21625R,12:30-3:50pm,Monday,6 of 11,Vince Gonzales,ANN211,6,11,Open


Looks like the section I wanted is closed. Oh well.

But now I can check frequently with the magic of Python and Pandas!